In [7]:
import pandas as pd
import os

In [8]:
REDEFINED_DIR = "../../../downloads/olist_redefined"
STREAM_DST = os.path.join(REDEFINED_DIR, 'stream')
BATCH_DST = os.path.join(REDEFINED_DIR, 'batch')
os.makedirs(STREAM_DST, exist_ok=True)
os.makedirs(BATCH_DST, exist_ok=True)

In [9]:
import pandas as pd

def get_unique_categories(products: pd.DataFrame, categories: pd.DataFrame) -> pd.DataFrame:
    """
    제품 데이터에서 실제 사용되는 카테고리와 카테고리 번역 정보를 left join하여 반환.
    """
    # 고유 카테고리 추출 및 Null 제거
    real_product_categories = products[['product_category_name']].drop_duplicates().dropna()
    
    # 카테고리 데이터와 left join
    unique_categories = real_product_categories.merge(
        categories,
        on='product_category_name',
        how='left'
    )
    return unique_categories

def fill_missing_category_names_manually(categories: pd.DataFrame) -> pd.DataFrame:
    """
    카테고리 영문명이 누락된 경우, 하드코딩 매핑값으로 채움.
    """
    categories['product_category_name_english'] = categories.apply(
        lambda row: 'gaming_pc' if row['product_category_name'] == 'pc_gamer'
        else 'portable_kitchen_and_food_preparators' if row['product_category_name'] == 'portateis_cozinha_e_preparadores_de_alimentos'
        else row['product_category_name_english'],
        axis=1
    )
    return categories

def make_column_rename_dict(columns: list) -> dict:
    """
    컬럼명 변경 규칙을 dict로 반환.
    - product_id는 그대로,
    - product_category_name_english는 'category'로,
    - 그 외는 첫 prefix 제외 + 오타(lenght → length) 보정
    """
    rename_dict = {}
    for c in columns:
        if c == 'product_id':
            rename_dict[c] = c
        elif c == 'product_category_name_english':
            rename_dict[c] = 'category'
        else:
            parts = c.split('_')[1:]
            # 오타 보정
            parts = ['length' if p == 'lenght' else p for p in parts]
            new_c = '_'.join(parts)
            rename_dict[c] = new_c
    return rename_dict

def rename_columns(df: pd.DataFrame, rename_dict: dict) -> pd.DataFrame:
    """
    컬럼명을 rename_dict 기준으로 일괄 변경
    """
    return df.rename(columns=rename_dict)

In [10]:
# 데이터 로드
products = pd.read_csv("../../../downloads/olist/olist_products_dataset.csv")
eng_categories = pd.read_csv("../../../downloads/olist/product_category_name_translation.csv")

In [11]:
# Step 1: 카테고리 매핑
unique_categories = get_unique_categories(products, eng_categories)

# Step 2: 누락된 카테고리 영문명 보정
null_unique_categories = unique_categories[unique_categories['product_category_name_english'].isna()]
if not null_unique_categories.empty:
    unique_categories = fill_missing_category_names_manually(unique_categories)
    print("[INFO] 누락된 카테고리 영문명을 보정하였습니다:")
    print(unique_categories[unique_categories['product_category_name_english'].isin(
        ['gaming_pc', 'portable_kitchen_and_food_preparators']
    )])

# Step 3: product 데이터와 조인
product_spec = products.merge(
    unique_categories,
    on='product_category_name',
    how='left'
).drop(columns='product_category_name')

# Step 4: 컬럼명 표준화
rename_dict = make_column_rename_dict(product_spec.columns)
product_spec = rename_columns(product_spec, rename_dict)

# Step 5: 결과 저장
product_spec.to_csv(f"{BATCH_DST}/products.tsv", index=False, sep='\t')

[INFO] 누락된 카테고리 영문명을 보정하였습니다:
                            product_category_name  \
63                                       pc_gamer   
69  portateis_cozinha_e_preparadores_de_alimentos   

            product_category_name_english  
63                              gaming_pc  
69  portable_kitchen_and_food_preparators  
